In [1]:
!pip install denoiser onnx onnxruntime

In [1]:
from IPython import display as disp
import torch
import torchaudio
from denoiser import pretrained
from denoiser.dsp import convert_audio

In [4]:
!wget https://facebookresearch.github.io/denoiser/audio/noisy/alex_noisy.mp3
!wget https://github.com/MohammadRaziei/libspeech/releases/download/resources/audio-fa-noisy.mp3

--2025-03-25 11:20:45--  https://facebookresearch.github.io/denoiser/audio/noisy/alex_noisy.mp3
Resolving facebookresearch.github.io (facebookresearch.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to facebookresearch.github.io (facebookresearch.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 339840 (332K) [audio/mp3]
Saving to: ‘alex_noisy.mp3.2’

alex_noisy.mp3.2    100%[===================>] 331.88K  --.-KB/s    in 0.05s   

2025-03-25 11:20:45 (6.36 MB/s) - ‘alex_noisy.mp3.2’ saved [339840/339840]

--2025-03-25 11:20:45--  https://github.com/MohammadRaziei/libspeech/releases/download/resources/audio-fa-noisy.mp3
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/948422393/1bdc893f-db93-412d-9fbb

In [19]:
model = pretrained._demucs(True, pretrained.DNS_64_URL, hidden=64, resample=4)
wav, sr = torchaudio.load('audio-fa-noisy.mp3')
wav = wav[:,:sr*10]
wav = convert_audio(wav, sr, model.sample_rate, model.chin)
with torch.no_grad():
    denoised = model(wav[None])[0]
disp.display(disp.Audio(wav.data.cpu().numpy(), rate=model.sample_rate))
disp.display(disp.Audio(denoised.data.cpu().numpy(), rate=model.sample_rate))

In [20]:
import torch

model = pretrained.dns64()

dummy_input = torch.randn(1, model.chin, 16000*32) # [batch_size, channels, length]

onnx_file_path = "dns64_model.onnx"
with torch.no_grad():
    torch.onnx.export(
        model,
        dummy_input,
        onnx_file_path,
        export_params=True,
        opset_version=13,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={
            "input": {0: "batch_size", 2: "length"},
            "output": {0: "batch_size", 2: "length"}
        }
    )

print(f"Model saved to {onnx_file_path}")

/usr/local/lib/python3.11/dist-packages/denoiser/demucs.py:146: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  length = math.ceil(length * self.resample)
/usr/local/lib/python3.11/dist-packages/denoiser/resample.py:19: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return th.where(t == 0, th.tensor(1., device=t.device, dtype=t.dtype), th.sin(t) / t)
/usr/local/lib/python3.11/dist-packages/denoiser/resample.py:67: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't reco

Model saved to dns64_model.onnx


In [21]:
import onnxruntime as ort
import numpy as np

onnx_file_path = "dns64_model2.onnx"  # مسیر فایل ONNX
ort_session = ort.InferenceSession(onnx_file_path)

input_name = ort_session.get_inputs()[0].name
dummy_input = wav.unsqueeze(0).cpu().numpy()  # ورودی نمونه

ort_outputs = ort_session.run(None, {input_name: dummy_input})

disp.display(disp.Audio(ort_outputs[0].squeeze(0), rate=model.sample_rate))

In [8]:
!cp dns64_model.onnx dns64_model2.onnx

In [ ]:
import torch

model = pretrained.dns48()

dummy_input = torch.randn(1, model.chin, 160000)  # [batch_size, channels, length]

onnx_file_path = "dns48_model.onnx"
with torch.no_grad():
    torch.onnx.export(
        model,
        dummy_input,
        onnx_file_path,
        export_params=True,
        opset_version=13,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={
            "input": {0: "batch_size", 2: "length"},
            "output": {0: "batch_size", 2: "length"}
        }
    )

print(f"Model saved to {onnx_file_path}")

In [14]:
model

BLSTM(
  (lstm): LSTM(1024, 1024, num_layers=2)
)

In [ ]:
model = pretrained.master64().cuda()

dummy_input = torch.randn(1, model.chin, 160000).cuda()  # [batch_size, channels, length]

onnx_file_path = "master64_model.onnx"
with torch.no_grad():
    torch.onnx.export(
        model,
        dummy_input,
        onnx_file_path,
        export_params=True,
        opset_version=13,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={
            "input": {0: "batch_size", 2: "length"},
            "output": {0: "batch_size", 2: "length"}
        }
    )

print(f"Model saved to {onnx_file_path}")

In [ ]:
model = pretrained.valentini_nc().cuda()

dummy_input = torch.randn(1, model.chin, 160000).cuda()  # [batch_size, channels, length]

onnx_file_path = "valentini_nc_model.onnx"
with torch.no_grad():
    torch.onnx.export(
        model,
        dummy_input,
        onnx_file_path,
        export_params=True,
        opset_version=13,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={
            "input": {0: "batch_size", 2: "length"},
            "output": {0: "batch_size", 2: "length"}
        }
    )

print(f"Model saved to {onnx_file_path}")